In [78]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.convolution import convolve, Box1DKernel
%matplotlib 

Using matplotlib backend: Qt5Agg


In [79]:
# importing fits files
file23 = fits.open('ch2_xsm_20210923_v1_level2.lc')
file24 = fits.open('ch2_xsm_20210924_v1_level2.lc')
file27 = fits.open('ch2_xsm_20210927_v1_level2.lc')
file28 = fits.open('ch2_xsm_20210928_v1_level2.lc')
file29 = fits.open('ch2_xsm_20210929_v1_level2.lc')

data23 = file23[1].data
data24 = file24[1].data
data27 = file27[1].data
data28 = file28[1].data
data29 = file29[1].data

In [80]:
data23

FITS_rec([(1.49126401e+08, 494.83276, 22.244837, 1.),
          (1.49126402e+08, 518.2688 , 22.765518, 1.),
          (1.49126403e+08, 504.4697 , 22.460403, 1.), ...,
          (1.49212797e+08, 376.00684, 19.390896, 1.),
          (1.49212798e+08, 374.12167, 19.342226, 1.),
          (1.49212799e+08, 401.34076, 20.033491, 1.)],
         dtype=(numpy.record, [('TIME', '>f8'), ('RATE', '>f4'), ('ERROR', '>f4'), ('FRACEXP', '>f4')]))

In [81]:
# checking how many data values in each fits file (1 day)

for data in [data23, data24, data27, data28, data29]:
    print(data[-1][0] - data[0][0])

86398.0
86247.0
86398.0
86399.0
86398.0


In [95]:
'''
    This function flare will take input parameters of manually shortlisting of flares from
    inspection of rate count plot vs time, and then in the same function, it will compute all other flares which
    can be missed manually. Manual flares and their times of start/stop were calculated by manual inspection and are present
    in lower cells
'''

def flare(data, num, t1, t2, fact):
    
# data is of the rate count and time from fits file
# num is number of flares we figured out manually
# t1 is the array of start time of manually figured flares and t2 is end time, so cardinality of t1, t2 is num 
# fact is the multiple we use ahead in this code to find the threshold of computing flare

    Rate = np.array(data['RATE']) # taking rate count from the data
    Time = np.array(data['TIME']) # taking time values
    
    # converting time to integer
    Time = Time.astype(int)
    
    # array for carrying the indexes of times when no (manual) flares are occuring, now it has all indexes but we will reduce it ahead
    time_bg_index = np.arange(0,len(Time))
    
    # array for carrying the indexes of times when (manual) flares are occuring, now it is empty but we will add into it
    flare_index = []
    
    t_start = t1
    t_stop = t2
    t_peak = [] # to store time values when peak rate value of (manual) flares
    f_mean = [] # to store the mean rate value of (manual) flares
    f_peak = [] # to store peak of rate of (manual) flares
    f_riseslope = []
    f_fallslope = []
    t_origin = data['TIME'][0] # origin of time for the file (offset) 
    
    n = num # number of (manual) flares
    print("Start and end time for manually chosen flares (offset time)")
    # filling all above arrays using this for loop
    for i in range(0,n):
        
        # start time and stop time for flare
        print("Flare ", i+1)
#         print("\n")
        print(np.where(Time == t_start[i])[0][0], "0")
#         print("\n")
        print(np.where(Time == t_stop[i])[0][0])
        print("\n")
        
        # calculating parameters of (manual) flares
        f_mean.append(np.average(Rate[np.where(Time == t_start[i])[0][0]:np.where(Time == t_stop[i])[0][0]]))
        f_peak.append(np.max(Rate[np.where(Time == t_start[i])[0][0]:np.where(Time == t_stop[i])[0][0]]))
        t_peak.append(Time[np.where(Rate == f_peak[i])[0][0]])
        f_riseslope.append((f_peak[i] - Rate[np.where(Time == t_start[i])[0][0]])/(max(t_peak[i] - t_start[i],1))
        f_fallslope.append((f_peak[i] - Rate[np.where(Time == t_stop[i])[0][0]])/(max(t_stop[i] - t_peak[i],1))
        
        
        # index of times when (manual) flare is occuring
        timeflare = np.intersect1d(np.where(Time > t_start[i])[0], np.where(Time < t_stop[i])[0])
        
        # index of times when no (manual) flares is occuring (this will exclude all the flares after loop is finished)
        time_bg_index = np.setdiff1d(time_bg_index, timeflare)
        
    # calculating indexes where flares are occuring by subtracting indexs of background from total
    flare_index = np.setdiff1d(np.arange(0,len(Time)), time_bg_index)
    
    # calculating some more parameters, mean of background, std dev of all flares, std dev of bg
    f_bg_mean = np.average(Rate[time_bg_index])
    f_stddev = np.std(Rate[flare_index])
    bg_stddev = np.std(Rate[time_bg_index])
    
    
    # plotting (manual) flares
    plt.figure(figsize=(7,4))
    plt.plot(data['TIME'],data['RATE'])
    plt.xlabel("Time")
    plt.ylabel("Rate")
    plt.vlines(x=t_start, ymin=0, ymax=4500, linestyles="--", color="Green") # star of flare -- Green
    plt.vlines(x=t_stop, ymin=0, ymax=4500, linestyles="--", color="Red") # stop of flare -- Red
    print("\n")
    
    
    # printing all the parameters of all the (manual) flares
    for i in range(0,n):
        print(f"Mean Rate count for flare {i+1} is {f_mean[i]}")
        print(f"Peak Rate count for flare {i+1} is {f_peak[i]}")
        print(f"Average slope of rise for flare {i+1} is {f_riseslope[i]}")
        print(f"Average slope of fall for flare {i+1} is {f_fallslope[i]}")
        print("\n")
      
    # printing background parameters
    print(f"Background mean rate is {f_bg_mean}")
    print(f"Std dev of Rate count for flares is: {f_stddev}")
    print(f"Std dev of bg Rate count is: {bg_stddev}")  
    
    
    # defining new arrays to store (computed) flares using the parameters calculated till now
    t_start2 = []
    t_stop2 = []
    t_peak2 = []
    f_mean2 = []
    f_peak2 = []
    f_riseslope2 = []
    f_fallslope2 = []
    
    flare = 0 # flag to store if flare is ON/OFF (1/0) while parsing the data file time (sec) wise
    s = fact # multiple of std dev to find the threshold of rate count to find the flare
    count = 0 # counts of flares
    time_buff_start = 0 # buffer time which is needed to confirm if flare has started
    time_buff_stop = 0 # buffer time which is needed to confirm if flare has stopped
    
    # threshold rate count to mark occurance of flare
    threshold = f_bg_mean + s*bg_stddev
    
    '''
        Computing the flares based on buffer time and threshold value, the multiple of std dev (s) is decided
        based on few trial values and manual inspection of the computed flares. buffer time is such that flare needed
        above/below the threshold for that much time to count of the flare
    '''
    
    # code to compute flares, for loop will go through entire array
    for i in range(0, len(Rate)):
        if Rate[i] > threshold and flare == 0 and time_buff_start > 12:
            t_start2.append(Time[i])
            flare = 1
            time_buff_stop = 0
            count += 1
        if Rate[i] < threshold and flare == 1 and time_buff_stop > 12:
            t_stop2.append(Time[i])
            flare = 0
            time_buff_start = 0
        if Rate[i] > threshold:
            time_buff_stop = 0
        else:
            time_buff_start = 0
        time_buff_stop += 1
        time_buff_start += 1
    
    # if flare is not ended at the end of array, count it the end
    if flare == 1:
        t_stop2.append(Time[len(Time)-1])
    
    # converting times to integer and printing
    t_start2 = [int(item) for item in t_start2]
    t_stop2 = [int(item) for item in t_stop2]
    print(t_start2)
    print("\n")
    print(t_stop2)
    
    # computing other parameters of the (computed) flares
    for i in range(0,count):
        f_mean2.append(np.average(Rate[np.where(Time == t_start2[i])[0][0]:np.where(Time == t_stop2[i])[0][0]]))
        f_peak2.append(np.max(Rate[np.where(Time == t_start2[i])[0][0]:np.where(Time == t_stop2[i])[0][0]]))
        t_peak2.append(Time[np.where(Rate == f_peak2[i])[0][0]])
        f_riseslope2.append((f_peak2[i] - Rate[np.where(Time == t_start2[i])[0][0]])/(max(t_peak2[i] - t_start2[i],1))
        f_fallslope2.append((f_peak2[i] - Rate[np.where(Time == t_stop2[i])[0][0]])/(max(t_stop2[i] - t_peak2[i],1))
        
    # plotting
    plt.figure(figsize=(7,4))
    plt.plot(data['TIME'],data['RATE'])
    plt.xlabel("Time")
    plt.ylabel("Rate")
    plt.vlines(x=t_start2, ymin=0, ymax=4500, linestyles="--", color="Green", label='Start time')
    plt.vlines(x=t_stop2, ymin=0, ymax=4500, linestyles="--", color="Red", label = 'Stop time')
    plt.hlines(y=threshold, xmin=Time[0], xmax=Time[len(Time)-1], linestyles="dashdot", color="Black", label='Threshold')
    plt.legend()
    plt.show()
    print("\n")
    
    
    print("Total flares from comptutation: ", count)
    
    # printing other parameters of the (computed) flares
    for i in range(0,count):
        print(f"Mean Rate count for flare {i+1} is {f_mean2[i]}")
        print(f"Peak Rate count for flare {i+1} is {f_peak2[i]}")
        print(f"Average slope of rise for flare {i+1} is {f_riseslope2[i]}")
        print(f"Average slope of fall for flare {i+1} is {f_fallslope2[i]}")
        print("\n")

SyntaxError: invalid syntax (<ipython-input-95-8ddbb7a9e186>, line 54)

In [98]:
# Same functioning as above but smoothing the light curve, except it has one more extra argument k (to convolve data)
def flaresmooth(data, num, t1, t2, fact, k):
    
    Rate = convolve(data['RATE'], Box1DKernel(10)) # convolving rate to make it smoother
    Time = np.array(data['TIME'])
    Time = Time.astype(int)
    time_bg_index = np.arange(0,len(Time))
    flare_index = []
    t_start = t1
    t_stop = t2
    t_peak = []
    f_mean = []
    f_peak = []
    f_riseslope = []
    f_fallslope = []
    t_origin = data['TIME'][0]
    n = num
    
    print("Start and end times of manually chosen flares (offset times)")
    for i in range(0,n):
        print("Flare ",i+1)
        print("Start Time: ", np.where(Time == t_start[i])[0][0])
        print("\n")
        print("Stop Time :",  np.where(Time == t_stop[i])[0][0])
        f_mean.append(np.average(Rate[np.where(Time == t_start[i])[0][0]:np.where(Time == t_stop[i])[0][0]]))
        f_peak.append(np.max(Rate[np.where(Time == t_start[i])[0][0]:np.where(Time == t_stop[i])[0][0]]))
        t_peak.append(Time[np.where(Rate == f_peak[i])[0][0]])
        f_riseslope.append((f_peak[i] - Rate[np.where(Time == t_start[i])[0][0]])/(max(t_peak[i] - t_start[i],1))
        f_fallslope.append((f_peak[i] - Rate[np.where(Time == t_stop[i])[0][0]])/(max(t_stop[i] - t_peak[i],1)))
        
        timeflare = np.intersect1d(np.where(Time > t_start[i])[0], np.where(Time < t_stop[i])[0])
        time_bg_index = np.setdiff1d(time_bg_index, timeflare)
    flare_index = np.setdiff1d(np.arange(0,len(Time)), time_bg_index)
    f_bg_mean = np.average(Rate[time_bg_index])
    f_stddev = np.std(Rate[flare_index])
    bg_stddev = np.std(Rate[time_bg_index])
    
    plt.figure(figsize=(7,4))
    plt.plot(data['TIME'],data['RATE'])
    plt.xlabel("Time")
    plt.ylabel("Rate")
    plt.vlines(x=t_start, ymin=0, ymax=4500, linestyles="--", color="Green")
    plt.vlines(x=t_stop, ymin=0, ymax=4500, linestyles="--", color="Red")
    print("\n")
    for i in range(0,n):
        
      
        print(f"Mean Rate count for flare {i+1} is {f_mean[i]}")
        print(f"Peak Rate count for flare {i+1} is {f_peak[i]}")
        print(f"Average slope of rise for flare {i+1} is {f_riseslope[i]}")
        print(f"Average slope of fall for flare {i+1} is {f_fallslope[i]}")
        print("\n")
      
    print(f"Background mean rate is {f_bg_mean}")
    print(f"Std dev of Rate count for flares is: {f_stddev}")
    print(f"Std dev of bg Rate count is: {bg_stddev}")  
    
    t_start2 = []
    t_stop2 = []
    t_peak2 = []
    f_mean2 = []
    f_peak2 = []
    f_riseslope2 = []
    f_fallslope2 = []
    k = k
    flare = 0
    s = fact
    count = 0
    time_buff_stop = 0
    time_buff_start = 0
    threshold = f_bg_mean + s*bg_stddev
    for i in range(0, len(Rate)):
        if Rate[i] > threshold + k*bg_stddev and flare == 0 and time_buff_start > 1:
            t_start2.append(Time[i])
            flare = 1
            time_buff_stop = 0
            count += 1
        if Rate[i] < threshold - k*bg_stddev and flare == 1 and time_buff_stop > 1:
            t_stop2.append(Time[i])
            flare = 0
            time_buff_start = 0
        if Rate[i] > threshold:
            time_buff_stop = 0
        else:
            time_buff_start = 0
        time_buff_stop += 1
        time_buff_start += 1
    if flare == 1:
        t_stop2.append(Time[len(Time)-1])
    t_start2 = [int(item) for item in t_start2]
    t_stop2 = [int(item) for item in t_stop2]
    print(t_start2)
    print("\n")
    print(t_stop2)
    for i in range(0,count):
        f_mean2.append(np.average(Rate[np.where(Time == t_start2[i])[0][0]:np.where(Time == t_stop2[i])[0][0]]))
        f_peak2.append(np.max(Rate[np.where(Time == t_start2[i])[0][0]:np.where(Time == t_stop2[i])[0][0]]))
        t_peak2.append(Time[np.where(Rate == f_peak2[i])[0][0]])
        f_riseslope2.append((f_peak2[i] - Rate[np.where(Time == t_start2[i])[0][0]])/(max(t_peak2[i] - t_start2[i],1))
        f_fallslope2.append((f_peak2[i] - Rate[np.where(Time == t_stop2[i])[0][0]])/(max(t_stop2[i] - t_peak2[i],1))
        
    plt.figure(figsize=(7,4))
    plt.plot(Time, Rate)
    plt.xlabel("Time")
    plt.ylabel("Rate")
    plt.vlines(x=t_start2, ymin=0, ymax=4500, linestyles="--", color="Green", label='Start time')
    plt.vlines(x=t_stop2, ymin=0, ymax=4500, linestyles="--", color="Red", label = 'Stop time')
    plt.hlines(y=threshold, xmin=Time[0], xmax=Time[len(Time)-1], linestyles="dashdot", color="Black", label='Threshold')
    plt.legend()
    plt.show()
    print("\n")
    print("Total computed flares", count)
    
    for i in range(0, count): 
      
        print(f"Mean Rate count for flare {i+1} is {f_mean2[i]}")
        print(f"Peak Rate count for flare {i+1} is {f_peak2[i]}")
        print(f"Average slope of rise for flare {i+1} is {f_riseslope2[i]}")
        print(f"Average slope of fall for flare {i+1} is {f_fallslope2[i]}")
        print("\n")

SyntaxError: invalid syntax (<ipython-input-98-8e74bc789f07>, line 29)

In [99]:
# Manually inspected flares starting and ending time points for each data files:

#data 23:
t11 = [1.491419e8, 1.491813e8]
t12 = [1.491458e8, 1.491845e8]

#data24:
t21 = [1.4923782e8, 1.4924878e8, 1.4927123e8, 1.4927747e8]
t22 = [1.4923947e8, 1.4924987e8, 1.4927276e8, 1.4928168e8]

#data27:
t31 = [1.4949891e8, 1.4950441e8, 1.4951411e8, 1.4951916e8, 1.4954254e8]
t32 = [1.4950370e8, 1.4950849e8, 1.4951878e8, 1.4952593e8, 1.4955518e8]

#data28:
t41 = [1.495779e8]
t42 = [1.495978e8]

#data29:
t51 = [1.496523e8, 1.496749e8, 1.496822e8, 1.496948e8, 1.497039e8, 1.497140e8, 1.497232e8, 1.497261e8]
t52 = [1.496545e8, 1.496815e8, 1.496942e8, 1.496971e8, 1.497055e8, 1.497208e8, 1.497260e8, 1.497282e8]

# flare(data23, 2, t11, t12, 5)
flaresmooth(data24, 4, t21, t22, 5, 0.3)

Start and end times of manually chosen flares (offset times)
24303


25953
35262


36352
57014


58544
63254


66762


Mean Rate count for flare 1 is 1362.9718017578125
Peak Rate count for flare 1 is 4773.98828125
Average slope of rise for flare 1 is 15.702816233915442
Average slope of fall for flare 1 is 3.078702220722968


Mean Rate count for flare 2 is 548.4730224609375
Peak Rate count for flare 2 is 730.352294921875
Average slope of rise for flare 2 is 0.6175501384432354
Average slope of fall for flare 2 is 0.4780095331497973


Mean Rate count for flare 3 is 601.3746948242188
Peak Rate count for flare 3 is 858.0097045898438
Average slope of rise for flare 3 is 0.6893591178928775
Average slope of fall for flare 3 is 0.409451893084906


Mean Rate count for flare 4 is 694.8172607421875
Peak Rate count for flare 4 is 1068.10595703125
Average slope of rise for flare 4 is 0.6268142842800818
Average slope of fall for flare 4 is 0.19210570511544586


Background mean rate is 439.71228027343

In [57]:
# this is one more method like we smoothed the light curve before. Here, we are just binning the rate counts and bin size is set as k

def flaresbin(data, num, t1, t2, fact, k):
    
    bin = 20
    
    Rate = data['RATE'][::bin]
    Time = np.array(data['TIME'][::bin])
    Time = Time.astype(int)
    time_bg_index = np.arange(0,len(Time))
    flare_index = []
    t_start = t1
    t_stop = t2
    t_peak = []
    f_mean = []
    f_peak = []
    f_riseslope = []
    f_fallslope = []
    t_origin = data['TIME'][0]
    n = num

    for i in range(0,n):
#         print(np.where(Time == t_start[i])[0][0])
#         print("\n")
#         print(np.where(Time == t_stop[i])[0][0])
        f_mean.append(np.average(Rate[np.where(abs(Time - t_start[i]) <= bin/2.0)[0][0]:np.where(abs(Time - t_stop[i]) <= bin/2.0)[0][0]]))
        f_peak.append(np.max(Rate[np.where(abs(Time - t_start[i]) <= bin/2.0)[0][0]:np.where(abs(Time - t_stop[i]) <= bin/2.0)[0][0]]))
        t_peak.append(Time[np.where(Rate == f_peak[i])[0][0]])
        f_riseslope.append((f_peak[i] - Rate[np.where(abs(Time - t_start[i]) <= bin/2.0)[0][0]])/(max(t_peak[i] - t_start[i],1))
        f_fallslope.append((f_peak[i] - Rate[np.where(abs(Time - t_start[i]) <= bin/2.0)[0][0]])/(max(t_stop[i] - t_peak[i],1))
        
        timeflare = np.intersect1d(np.where(Time > t_start[i])[0], np.where(Time < t_stop[i])[0])
        time_bg_index = np.setdiff1d(time_bg_index, timeflare)
    flare_index = np.setdiff1d(np.arange(0,len(Time)), time_bg_index)
    f_bg_mean = np.average(Rate[time_bg_index])
    f_stddev = np.std(Rate[flare_index])
    bg_stddev = np.std(Rate[time_bg_index])
    
    plt.figure(figsize=(7,4))
    plt.plot(data['TIME'],data['RATE'])
    plt.xlabel("Time")
    plt.ylabel("Rate")
    plt.vlines(x=t_start, ymin=0, ymax=4500, linestyles="--", color="Green")
    plt.vlines(x=t_stop, ymin=0, ymax=4500, linestyles="--", color="Red")
    print("\n")
    for i in range(0,n):
        
      
        print(f"Mean Rate count for flare {i+1} is {f_mean[i]}")
        print(f"Peak Rate count for flare {i+1} is {f_peak[i]}")
        print(f"Average slope of rise for flare {i+1} is {f_riseslope[i]}")
        print(f"Average slope of fall for flare {i+1} is {f_fallslope[i]}")
        print("\n")
      
    print(f"Background mean rate is {f_bg_mean}")
    print(f"Std dev of Rate count for flares is: {f_stddev}")
    print(f"Std dev of bg Rate count is: {bg_stddev}")  
    
    t_start2 = []
    t_stop2 = []
    t_peak2 = []
    f_mean2 = []
    f_peak2 = []
    f_riseslope2 = []
    f_fallslope2 = []
    time_bg_index2 = np.arange(0,len(Time))
    k = k
    flare = 0
    s = fact
    count = 0
    time_buff_stop = 0
    time_buff_start = 0
    threshold = f_bg_mean + s*bg_stddev
    for i in range(0, len(Rate)):
        if Rate[i] > threshold + k*bg_stddev and flare == 0 and time_buff_start > 1:
            t_start2.append(Time[i])
            flare = 1
            time_buff_stop = 0
            count += 1
        if Rate[i] < threshold - k*bg_stddev and flare == 1 and time_buff_stop > 1:
            t_stop2.append(Time[i])
            flare = 0
            time_buff_start = 0
        if Rate[i] > threshold:
            time_buff_stop = 0
        else:
            time_buff_start = 0
        time_buff_stop += 1
        time_buff_start += 1
    if flare == 1:
        t_stop2.append(Time[len(Time)-1])
    t_start2 = [int(item) for item in t_start2]
    t_stop2 = [int(item) for item in t_stop2]
    print(t_start2)
    print("\n")
    print(t_stop2)
    for i in range(0,count):
        f_mean2.append(np.average(Rate[np.where(abs(Time - t_start2[i]) <= bin/2.0)[0][0]:np.where(abs(Time - t_stop2[i]) <= bin/2.0)[0][0]]))
        f_peak2.append(np.max(Rate[np.where(abs(Time - t_start2[i]) <= bin/2.0)[0][0]:np.where(abs(Time - t_stop2[i]) <= bin/2.0)[0][0]]))
        t_peak2.append(Time[np.where(Rate == f_peak2[i])[0][0]])
        f_riseslope2.append((f_peak2[i] - Rate[np.where(abs(Time - t_start2[i]) <= bin/2.0)[0][0]])/(max(t_peak2[i] - t_start2[i],1))
        f_fallslope2.append((f_peak2[i] - Rate[np.where(abs(Time - t_stop2[i]) <= bin/2.0)[0][0]])/(max(t_stop2[i] - t_peak2[i],1))
    
        timeflare2 = np.intersect1d(np.where(Time > t_start2[i])[0], np.where(Time < t_stop2[i])[0])
        time_bg_index2 = np.setdiff1d(time_bg_index2, timeflare2)
    
    flare_index2 = np.setdiff1d(np.arange(0,len(Time)), time_bg_index2)
    f_bg_mean2 = np.average(Rate[time_bg_index2])
    f_stddev2 = np.std(Rate[flare_index2])
    bg_stddev2 = np.std(Rate[time_bg_index2])
    
   
    print("\n")
    print("Total computed flares:", count)
    for i in range(0,count):
        
        print(f"Mean Rate count for flare {i+1} is {f_mean2[i]}")
        print(f"Peak Rate count for flare {i+1} is {f_peak2[i]}")
        print(f"Average slope of rise for flare {i+1} is {f_riseslope2[i]}")
        print(f"Average slope of fall for flare {i+1} is {f_fallslope2[i]}")
        print("\n")
    print(f"Background mean rate is {f_bg_mean2}")
    print(f"Std dev of Rate count for flares is: {f_stddev2}")
    print(f"Std dev of bg Rate count is: {bg_stddev2}")  
    
    plt.figure(figsize=(7,4))
    plt.plot(Time, Rate)
    plt.xlabel("Time")
    plt.ylabel("Rate")
    plt.vlines(x=t_start2, ymin=0, ymax=30000, linestyles="--", color="Green", label='Start time')
    plt.vlines(x=t_stop2, ymin=0, ymax=30000, linestyles="--", color="Red", label = 'Stop time')
    plt.hlines(y=threshold, xmin=Time[0], xmax=Time[len(Time)-1], linestyles="dashdot", color="Black", label='Threshold')
    plt.hlines(y = f_bg_mean2, xmin=Time[0], xmax=Time[len(Time)-1], linestyles="dotted", color="blue", label='background')
    
    plt.legend()
    plt.show()

In [58]:
#data 23:
t11 = [1.4914151e8, 1.4918177e8]
t12= [1.4914769e8, 1.4918762e8]

#data24:
t21 = [1.4923782e8, 1.4924878e8, 1.4927123e8, 1.4927747e8]
t22 = [1.4923947e8, 1.4924987e8, 1.4927276e8, 1.4928168e8]

#data27:
t31 = [1.4949891e8, 1.4950441e8, 1.4951411e8, 1.4951916e8, 1.4954254e8]
t32 = [1.4950370e8, 1.4950849e8, 1.4951878e8, 1.4952593e8, 1.4955518e8]

#data28:
t41 = [1.495779e8]
t42 = [1.495978e8]

#data29:
t51 = [1.496523e8, 1.496749e8, 1.496822e8, 1.496948e8, 1.497039e8, 1.497140e8, 1.497232e8, 1.497261e8]
t52 = [1.496545e8, 1.496815e8, 1.496942e8, 1.496971e8, 1.497055e8, 1.497208e8, 1.497260e8, 1.497282e8]

# flare(data24, 4, t21, t22, 5)
flaresbin(data23, 2, t11, t12, 2, 0.5)



Mean Rate count for flare 1 is 6219.68310546875
Peak Rate count for flare 1 is 31762.318359375
Average slope of rise for flare 1 is 16.710033643545795
Average slope of fall for flare 1 is 7.233395041154648


Mean Rate count for flare 2 is 3228.26904296875
Peak Rate count for flare 2 is 23325.005859375
Average slope of rise for flare 2 is 58.01429768249354
Average slope of fall for flare 2 is 4.10974431688175


Background mean rate is 577.5077514648438
Std dev of Rate count for flares is: 7046.62451171875
Std dev of bg Rate count is: 173.9422607421875
[149142417, 149155697, 149172217, 149176397, 149181157, 149200838]


[149146817, 149155857, 149172817, 149177017, 149186717, 149201258]


Total computed flares: 6
Mean Rate count for flare 1 is 8434.990234375
Peak Rate count for flare 1 is 31762.318359375
Average slope of rise for flare 1 is 32.04401761274765
Average slope of fall for flare 1 is 8.994509674781977


Mean Rate count for flare 2 is 932.0921630859375
Peak Rate count for flar